In [1]:
import os
import re
import csv
import time 
import random 
import sqlite3
import keyboard 
import funcionesAux as fc
from datetime import datetime
from bs4 import BeautifulSoup
from seleniumbase import Driver
from selenium.webdriver.common.by import By

def mercadona_csv(datos, nombre_archivo = "dia.csv"):
    """
    Guarda los datos en un archivo CSV.

    Parámetros:
    datos -- Lista de diccionarios con los datos a guardar.
    nombre_archivo -- Nombre del archivo CSV a crear.
    """

    # Nombres de las columnas
    columnas = datos[0].keys()

    # Comprueba si el archivo existe
    existe_archivo = os.path.isfile(nombre_archivo)

    # Abre el archivo en modo 'a+' (append) si existe, o en modo 'w' (write) si no existe
    with open(nombre_archivo, 'a+' if existe_archivo else 'w', newline='') as f:
        writer = csv.DictWriter(f, fieldnames=columnas)

        # Escribe el encabezado solo si el archivo no existía
        if not existe_archivo:
            writer.writeheader()

        # Escribe los datos en el archivo CSV
        writer.writerows(datos)

driver = Driver(
    browser="chrome",
    uc=True,
    headless2=False,
    incognito=False,
    agent='Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36',
    do_not_track=True,
    undetectable=True
)

driver.maximize_window()

fecha = datetime.now().date()
print(f"Iniciando escaneo a fecha: {datetime.now()}")
driver.get("https://tienda.mercadona.es/")
# Acepta cookies
fc.click_element(driver, By.XPATH, "//button[normalize-space()='Aceptar']")
time.sleep(3)
# Navega a la sección por categorías (estructura muy similar al Dia)
driver.get("https://tienda.mercadona.es/categories/112")

Iniciando escaneo a fecha: 2024-05-04 16:56:11.747651


In [48]:
categorias = fc.wait_for_elements(driver, By.CSS_SELECTOR, '.category-menu__header', multiple=True)
lista_categorias = []

for categoria in categorias: 
    category = categoria.text.replace(",","")
    lista_categorias.append(categoria.text)
    print(f"\nAnalizando: {category}")
    time.sleep(random.uniform(1,3))
    categoria.click()
    time.sleep(random.uniform(1,3))
    # Una vez estoy dentro de la categoría, como hay problemas, elijo category-open
    el_category = fc.wait_for_elements(driver, By.CSS_SELECTOR, 'li.category-menu__item.open', multiple=False)
    time.sleep(random.uniform(2,3))
    subcategorias = fc.wait_for_elements(el_category, By.CSS_SELECTOR, 'ul > li.category-item', multiple=True)
    for sub in subcategorias:
        print(sub.text)
        time.sleep(random.uniform(1,3))
        sub.click()
        time.sleep(random.uniform(1,3))

        # Obtengo el HTML del contenedor de productos 
        elemento_productos = fc.wait_for_elements(driver, By.CSS_SELECTOR, 'div.product-cell[data-testid="product-cell"]', multiple=True)
        print(f"Total productos: {len(elemento_productos)}")
        
        for anuncio in elemento_productos: 
            #time.sleep(random.uniform(1,3))
            html_content = anuncio.get_attribute('innerHTML')  # Esto te dará el HTML del elemento
            soup = BeautifulSoup(html_content, 'html.parser')
            img_element = soup.find('img')
            imagen = img_element['src']
            h4_element = soup.find('h4', class_="subhead1-r product-cell__description-name", attrs={"data-testid": "product-cell-name"})
            titulo = h4_element.text
            p_element = soup.find('p', class_="product-price__unit-price subhead1-b", attrs={"data-testid": "product-price"})
            if p_element is not None: 
                #p_element = soup.find('p', class_="product-price__unit-price subhead1-b", attrs={"data-testid": "product-price"})
                precio = p_element.text.replace(".","").replace(",",".").replace("€","").strip()
            else:
                p_element = soup.find('p', class_="product-price__unit-price subhead1-b product-price__unit-price--discount", attrs={"data-testid": "product-price"})
                precio = p_element.text.replace(".","").replace(",",".").replace("€","").strip()
            print(f"Titulo: {titulo}\nImagen: {imagen}\nPrecio: {precio}")

    # except: 
    #     print(f"Error al seleccionar {category}")
    


Analizando: Aceite especias y salsas


Aceite, vinagre y sal
36
Titulo: Aceite de oliva 0,4º Hacendado
Imagen: https://prod-mercadona.imgix.net/images/1a84cd7052b68873985104ac24b87043.jpg?fit=crop&h=300&w=300
Precio: 8.95
Titulo: Aceite de oliva virgen extra Hacendado
Imagen: https://prod-mercadona.imgix.net/images/f59ef64a2984bb74e7bfa40270605438.jpg?fit=crop&h=300&w=300
Precio: 29.55
Titulo: Aceite de oliva virgen extra Hacendado
Imagen: https://prod-mercadona.imgix.net/images/e19dbbdcfbd3e756081491ab7b676ffd.jpg?fit=crop&h=300&w=300
Precio: 9.90
Titulo: Aceite de oliva virgen extra Hacendado Gran Selección
Imagen: https://prod-mercadona.imgix.net/images/ca971ebf519d62b535e1d0ad05ebd394.jpg?fit=crop&h=300&w=300
Precio: 8.10
Titulo: Aceite de oliva 1º Hacendado
Imagen: https://prod-mercadona.imgix.net/images/b57ae00beedb23ec86686fa3651fd448.jpg?fit=crop&h=300&w=300
Precio: 8.95
Titulo: Aceite de oliva virgen Hacendado
Imagen: https://prod-mercadona.imgix.net/images/e58a263f9514576e38a96724b3104f9b.jpg?fit=crop&h=300&w=300


KeyboardInterrupt: 